---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from datetime import datetime, timedelta

In [31]:
train= pd.read_csv('train.csv', encoding = 'ISO-8859-1')
test= pd.read_csv('test.csv')

train = train[pd.notnull(train['hearing_date'])]
train = train[pd.notnull(train['ticket_issued_date'])]
train = train[pd.notnull(train['compliance'])]

#train ['ticket_issued_date'].dropna(inplace = True)
#train['compliance'].dropna(inplace=True)

X_train = train.drop('compliance', axis= 1)
y_train = train['compliance']

X_train.drop(['payment_status','payment_amount', 'payment_date','compliance_detail', 'balance_due', 'collection_status', 'violation_zip_code', 'grafitti_status', 'violation_description'
             ] , axis=1, inplace=True)

test.drop(['violation_zip_code', 'grafitti_status', 'violation_description'], axis=1, inplace=True)
#assert(X_train.shape[1]==test.shape[1])

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
X_train.shape, test.shape

((159653, 24), (61001, 24))

In [4]:
#X_train['hearing_date'].dropna(inplace= True)

In [33]:
X_train['In MI'] = X_train['state']=='MI'
X_train['In Detroit'] = X_train['city'] == 'DETROIT'
X_train['USA'] = X_train['country'] == 'USA'

In [36]:
test['In MI'] = test['state']=='MI'
test['In Detroit'] = test['city'] == 'DETROIT'
#test['USA'] = test['country'] == 'USA'

In [37]:
X_train['In Detroit']=  pd.get_dummies(X_train['In Detroit'], drop_first= True)
X_train['In MI'] = pd.get_dummies(X_train['In MI'],drop_first= True)
X_train['USA'] = pd.get_dummies(X_train['USA'], drop_first= True)

In [38]:
test['In Detroit']=  pd.get_dummies(test['In Detroit'], drop_first= True)
test['In MI'] = pd.get_dummies(test['In MI'],drop_first= True)
#test['USA'] = pd.get_dummies(test['USA'], drop_first= True)
test['USA'] =1

In [41]:
X_train.shape, test.shape

((159653, 27), (61001, 27))

In [42]:
disposition = pd.get_dummies(X_train['disposition'],drop_first=True)
agency = pd.get_dummies(X_train['agency_name'], drop_first= True)
violation_code = pd.get_dummies(X_train['violation_code'], drop_first= True)

In [43]:
disposition_t = pd.get_dummies(test['disposition'],drop_first=True)
agency_t = pd.get_dummies(test['agency_name'], drop_first= True)
violation_code_t = pd.get_dummies(test['violation_code'], drop_first= True)

In [44]:
disposition_t.drop(['Responsible - Compl/Adj by Determi', 'Responsible (Fine Waived) by Deter', 'Responsible - Compl/Adj by Default', 'Responsible by Dismissal'], axis=1, inplace=True)

In [45]:
for i in violation_code.columns:
    if i not in violation_code_t.columns:
        violation_code.drop(i, axis=1, inplace=True)
        
for i in violation_code_t.columns:
    if i not in violation_code.columns:
        violation_code_t.drop(i, axis=1, inplace=True)
        
for i in agency.columns:
    if i not in agency_t.columns:
        agency.drop(i, axis=1, inplace=True)

In [46]:
X_train = pd.concat([X_train, disposition, agency, violation_code], axis=1)

In [47]:
test = pd.concat([test, disposition_t, agency_t, violation_code_t], axis=1)

In [48]:
X_train.shape, test.shape

((159653, 148), (61001, 148))

In [16]:
violation_code.shape, violation_code_t.shape

((159653, 116), (61001, 116))

In [17]:
disposition_t.shape, disposition.shape

((61001, 3), (159653, 3))

In [18]:
agency.shape, agency_t.shape

((159653, 2), (61001, 2))

In [49]:
X_train['Difference in Time'] = (pd.to_datetime(X_train['hearing_date'], errors='coerce') -\
            pd.to_datetime(X_train['ticket_issued_date'], errors='coerce')).astype('timedelta64[M]').astype('float64')

In [50]:
test['Difference in Time'] = (pd.to_datetime(test['hearing_date'], errors='coerce') -\
            pd.to_datetime(test['ticket_issued_date'], errors='coerce')).astype('timedelta64[M]').astype('float64')

In [51]:
X_train.shape, test.shape

((159653, 149), (61001, 149))

In [52]:
X_train['Difference in Time'].fillna(0, inplace= True)

In [53]:
test['Difference in Time'].fillna(0, inplace= True)

In [54]:
X_train.drop(['agency_name', 'hearing_date', 'ticket_issued_date', 'inspector_name','violation_code', 'violation_street_number' ,'violation_street_name', 'violator_name', 'country', 'state', 'city', 'mailing_address_str_number','mailing_address_str_name' ,'zip_code', 'non_us_str_code', 'disposition', 'agency_name'
             ] , axis=1, inplace=True)

test.drop(['agency_name', 'hearing_date', 'ticket_issued_date', 'inspector_name','violation_code', 'violation_street_number' ,'violation_street_name', 'violator_name', 'country', 'state', 'city', 'mailing_address_str_number','mailing_address_str_name' ,'zip_code', 'non_us_str_code', 'disposition', 'agency_name'
             ] , axis=1, inplace=True)

In [55]:
X_train.shape, test.shape

((159653, 133), (61001, 133))

In [24]:
#X_train.dropna(axis=0, inplace=True)

In [25]:
#X_train = X_train.reset_index()

In [56]:
y_train.shape

(159653,)

In [57]:
test.shape

(61001, 133)

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [59]:
log_clf = LogisticRegression().fit(X_train, y_train)

In [60]:
prob = log_clf.predict_proba(test)

In [70]:
a = pd.Series(prob[:,1], index= test['ticket_id'])

In [72]:
a.shape

(61001,)

In [73]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return a

In [74]:
blight_model()

ticket_id
284932    5.530222e-02
285362    4.320145e-03
285361    7.469473e-02
285338    5.520483e-02
285346    7.482509e-02
285345    5.520455e-02
285347    8.685148e-02
285342    1.859797e-01
285530    4.330089e-03
284989    2.166228e-02
285344    8.674607e-02
285343    4.327057e-03
285340    4.320217e-03
285341    8.662027e-02
285349    7.488346e-02
285348    5.524853e-02
284991    2.166225e-02
285532    2.153861e-02
285406    2.154062e-02
285001    2.158419e-02
285006    4.329505e-03
285405    4.330497e-03
285337    2.165670e-02
285496    8.697526e-02
285497    5.535546e-02
285378    4.343560e-03
285589    2.158910e-02
285585    5.511216e-02
285501    7.489533e-02
285581    4.319432e-03
              ...     
376367    9.016286e-03
376366    4.412168e-02
376362    4.490661e-02
376363    5.241782e-02
376365    9.016299e-03
376364    4.412175e-02
376228    4.432035e-02
376265    4.431916e-02
376286    2.019596e-01
376320    4.419031e-02
376314    4.412334e-02
376327    2.014707e-01
3